In [47]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


def preprocess(df, num_cols, bin_cols, multi_cols, verbose=True):
    """
    Preprocess the DF for ML
    Args:
        df(pd.DataFrame): dataset you are working on
        num_cols(list): list of all numerical features
        bin_cols(list): list of all binary features
        multi_cols(list): list of all multicategorical features
        verbose(string): if you want to print a head on the final DF
    Returns:
        None
    Example:
    telcom = preprocess(telcom.copy(), num_cols, bin_cols, verbose=True)    
    """    
    #Label encoding Binary columns, which means if two values M, F -> 0, 1
    # here is to convert object type to int
    le = LabelEncoder()
    for i in bin_cols :
        df[i] = le.fit_transform(df[i])
    
    #Duplicating columns for multi value columns
    df = pd.get_dummies(data = df,columns = multi_cols )

    # Scaling Numerical columns
    std = StandardScaler()
    scaled = std.fit_transform(df[num_cols])
    scaled = pd.DataFrame(scaled,columns=num_cols)

    # dropping original values merging scaled values for numerical columns
    df = df.drop(columns = num_cols,axis = 1)
    df = df.merge(scaled,left_index=True,right_index=True,how = "left")
    
    df.columns = map(str.lower, df.columns)

    if verbose == True:
        print(df.head())
    
    return df



def data_preprocessing(dataset):
    """
    Bank
    """
    if dataset=="Bank":
        target_col = ["Exited"]

        df = pd.read_csv('/home/ec2-user/SageMaker/data/churn_package/datasets/Bank-data/Churn_Modelling.csv', sep=",")

        cat_cols   = df.nunique()[df.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(df.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in df.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- Unique preprocessing for this data set --------- #


        del df["CustomerId"]
        del df["Surname"]

        multi_cols = ['NumOfProducts', 'Geography']
        num_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

        # --------- general preprocessing --------- #

        df = preprocess(df, num_cols, bin_cols, multi_cols, verbose=False)

        # rename the target variable to Churn
        df.rename(columns={"exited":"churn"}, inplace=True)

        target_col = ["churn"]

        del df["rownumber"]
        return df

        
    elif dataset=="UCI":
        """
        UCI
        """
        target_col = ["churn"]
        df = pd.read_csv('/home/ec2-user/SageMaker/data/churn_package/datasets/UCI/UCI.csv', sep=",")

        cat_cols   = df.nunique()[df.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(df.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in df.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]


        del df["state"]

        del df["phone number"]
        multi_cols = ['area code']


        df = preprocess(df, num_cols, bin_cols, multi_cols, verbose=False)
        return df
    
    elif dataset == 'Mobile':        
        df = pd.read_csv('/home/ec2-user/SageMaker/data/churn_package/datasets/mobile-churn/mobile-churn-data.csv', sep=",")

        target_col = ["churn"]

        # ----------- Categorization ----------- #

        cat_cols   = df.nunique()[df.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(df.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in df.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]


        del df["year"]
        del df["month"]
        del df["user_account_id"]

        cat_cols   = df.nunique()[df.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(df.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in df.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        df[multi_cols] = df[multi_cols].apply(lambda x: x.str.replace(',','.'))
        df[multi_cols] = df[multi_cols].apply(pd.to_numeric)

        num_cols = num_cols + multi_cols

        multi_cols = []
        df = preprocess(df.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        # une fois que j'ai changer la virgule en point, j'ai eu masse NaN bizarre
        df = df.fillna(df.mean())
        return df
    
    elif dataset == 'KKBox':
        
        # ------ input ------ #        
        dataset_name_train = '/home/ec2-user/SageMaker/data/churn_package/datasets/KKBox/train_v2.csv'
        dataset_name_transaction = '/home/ec2-user/SageMaker/data/churn_package/datasets/KKBox/transactions_v2.csv'
        dataset_name_log = '/home/ec2-user/SageMaker/data/churn_package/datasets/KKBox/user_logs_v2.csv'
        dataset_name_mem = '/home/ec2-user/SageMaker/data/churn_package/datasets/KKBox/members_v3.csv'

        sep = ","

        # data preprocessing
        target_col = ["Churn"]

        # data for writing file
        dataset = "KKBox"

        train = pd.read_csv(dataset_name_train)
        transa = pd.read_csv(dataset_name_transaction)
        log = pd.read_csv(dataset_name_log)
        member = pd.read_csv(dataset_name_mem)

        train = pd.merge(train, member, on="msno", how="left")
        del member

        train = pd.merge(train,transa,how='left',on='msno',left_index=True, right_index=True)
        del transa

        train = pd.merge(train,log,how='left',on='msno',left_index=True, right_index=True)
        del log

        train['registration_init_time'] = train['registration_init_time'].fillna(value='20151009')
        train["transaction_date"] = pd.to_datetime(train["transaction_date"])
        train["date"] = pd.to_datetime(train["date"])
        train["membership_expire_date"] = pd.to_datetime(train["membership_expire_date"])
        train["registration_init_time"] = pd.to_datetime(train["registration_init_time"])

        def date_feature(df):

            col = ['registration_init_time' ,'transaction_date','membership_expire_date','date']
            var = ['reg','trans','mem_exp','user_']
            #df['duration'] = (df[col[1]] - df[col[0]]).dt.days 

            for i ,j in zip(col,var):
                df[j+'_day'] = df[i].dt.day.astype('uint8')
                df[j+'_weekday'] = df[i].dt.weekday.astype('uint8')        
                df[j+'_month'] = df[i].dt.month.astype('uint8') 
                df[j+'_year'] =df[i].dt.year.astype('uint16') 

        date_feature(train)

        col = [ 'city', 'bd', 'gender', 'registered_via']
        def missing(df,columns):
            col = columns
            for i in col:
                df[i].fillna(df[i].mode()[0],inplace=True)

        missing(train,col)

        le = LabelEncoder()
        train['gender'] = le.fit_transform(train['gender'])

        def OHE(df):
            #col = df.select_dtypes(include=['category']).columns
            col = ['city','gender','registered_via']
            print('Categorical columns in dataset',col)

            c2,c3 = [],{}
            for c in col:
                if df[c].nunique()>2 :
                    c2.append(c)
                    c3[c] = 'ohe_'+c

            df = pd.get_dummies(df,columns=c2,drop_first=True,prefix=c3)
            
            return df
        train1 = OHE(train)

        unwanted = ['msno','registration_init_time','transaction_date','membership_expire_date','date']

        train1.drop(unwanted,axis=1, inplace=True)
        # rename the target variable to Churn
        train1.rename(columns={"is_churn":"churn"}, inplace=True)
        return train1
    
    elif dataset == "K2009":
        
        # ------ input ------ #
        # Data file opening
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/Kdd2009-small/orange_small_train.data'
        target_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/Kdd2009-small/orange_small_train_churn.labels'
        sep = "\t"

        # data preprocessing
        target_col = ["churn"]

        # data for writing file
        dataset = "KDD-cup-2009-small"

        telcom = pd.read_csv(dataset_name, sep="\t")
        target = pd.read_csv(target_name, sep="\t")

        target.columns = ["churn"]

        # merge training set with its target
        telcom = telcom.join(target)

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]


        # list of columns
        old_col_telcom = telcom.columns


        # ----------- Preprocessing ----------- #

        # last observation is -0.788 for the churn so we remove it
        telcom = telcom[:-1]

        del telcom["churn"]

        # data cleaning: drop the columns with std close to zero
        # delete columns with at least 20% missing values
        threshold = 0.2
        telcom = telcom.drop(telcom.std()[telcom.std() < threshold].index.values, axis=1)
        telcom = telcom.loc[:, pd.notnull(telcom).sum() > len(telcom)*.8]

        DataVars = telcom.columns
        data_types = {Var: telcom[Var].dtype for Var in DataVars}

        for Var in DataVars:
            if data_types[Var] == int:
                x = telcom[Var].astype(float)
                telcom.loc[:, Var] = x
                data_types[Var] = x.dtype
            elif data_types[Var] != float:
                x = telcom[Var].astype('category')
                telcom.loc[:, Var] = x
                data_types[Var] = x.dtype

        # numerical data
        float_DataVars = [Var for Var in DataVars
                             if data_types[Var] == float]

        float_x_means = telcom.mean()

        for Var in float_DataVars:
            x = telcom[Var]
            isThereMissing = x.isnull()
            if isThereMissing.sum() > 0:
                telcom.loc[isThereMissing.tolist(), Var] = float_x_means[Var]     

        DataVars = telcom.columns

        categorical_DataVars = [Var for Var in DataVars
                                   if data_types[Var] != float]

        categorical_levels = telcom[categorical_DataVars].apply(lambda col: len(col.cat.categories))

        categorical_DataVars = categorical_levels[categorical_levels <= 500].index

        col_to_keep = float_DataVars + list(categorical_DataVars)

        telcom = telcom[col_to_keep]

        collapsed_categories = {}

        removed_categorical_DataVars = set()

        for Vars in categorical_DataVars:

            isTheremissing_value = telcom[Vars].isnull()
            if isTheremissing_value.sum() > 0:
                telcom[Vars].cat.add_categories('unknown', inplace=True)
                telcom.loc[isTheremissing_value.tolist(), Vars] = 'unknown'

        cat_cols = telcom.select_dtypes("category").columns
        num_cols = telcom.select_dtypes("float").columns

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # merge training set with its target
        telcom = telcom.join(target)

        # changing labels to 0 or 1
        telcom["churn"] = (telcom["churn"] +1)/2

        # convert to int the target variable othw it is 1.0 and 0.0
        telcom.churn = pd.to_numeric(telcom.churn, downcast='integer')

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        return telcom

    elif dataset == "Member":
        
        # ------ input ------ #
        # Data file opening
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/membershipWoes/Assignment- Membership woes.csv'
        sep = ","

        # data preprocessing
        target_col = ["Churn"]

        # data for writing file
        dataset = "membership-woes"

        telcom = pd.read_csv(dataset_name, sep)

        # --------- Unique preprocessing for this data set --------- #

        telcom.rename(columns = {'MEMBERSHIP_STATUS': "Churn"}, inplace=True)
        telcom["Churn"].map({'INFORCE': 0, 'CANCELLED': 1})

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- Unique preprocessing for this data set --------- #

        # preprocessing
        del telcom['START_DATE (YYYYMMDD)']
        del telcom['MEMBERSHIP_NUMBER']
        del telcom["AGENT_CODE"]
        del telcom['END_DATE  (YYYYMMDD)']

        num_cols = ['MEMBERSHIP_TERM_YEARS',
         'ANNUAL_FEES',
         'MEMBER_ANNUAL_INCOME',
         'MEMBER_AGE_AT_ISSUE']

        multi_cols =  ['ADDITIONAL_MEMBERS',
         'MEMBER_OCCUPATION_CD',
         'PAYMENT_MODE',
         'MEMBER_MARITAL_STATUS']

        # replace nan by most frequent value
        telcom["MEMBER_OCCUPATION_CD"].fillna(telcom["MEMBER_OCCUPATION_CD"].mode().iloc[0], inplace=True)
        telcom["MEMBER_GENDER"].fillna(telcom["MEMBER_GENDER"].mode().iloc[0], inplace=True)
        telcom["MEMBER_MARITAL_STATUS"].fillna(telcom["MEMBER_MARITAL_STATUS"].mode().iloc[0], inplace=True)

        # for numerical values replace nan by mean

        telcom.fillna(telcom.mean(), inplace=True)

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        return telcom
    
    
    elif dataset == "TelE":

        # ------ input ------ #
        # Data file opening
        path = "/Churn/datasets"
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/churn-telco-europa/train_churn_kg.csv'

        sep = ","
        

        # data preprocessing
        target_col = ["CHURN"]

        # data for writing file
        dataset = "telco-europa"

        telcom = pd.read_csv(dataset_name, sep)

        # --------- Unique preprocessing for this data set --------- #

        # None

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- Unique preprocessing for this data set --------- #

        # write report

        # --------- general preprocessing --------- #

        del telcom["CETEL_NUMBER"]
        del telcom["CNI_CUSTOMER"]


        num_cols = ['DAYS_LIFE',
         'DEVICE_TECNOLOGY',
         'MIN_PLAN',
         'PRICE_PLAN',
         'TOT_MIN_CALL_OUT',
         'AVG_MIN_CALL_OUT_3',
         'TOT_MIN_IN_ULT_MES',
         'AVG_MIN_IN_3',
         'ROA_LASTMONTH',
         'ROACETEL_LAST_MONTH',
         'DEVICE',
         'STATE_DATA',
         'CITY_DATA',
         'STATE_VOICE',
         'CITY_VOICE']

        multi_cols = ['TEC_ANT_DATA', 'TEC_ANT_VOICE']

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)

        target_col = ["churn"]

        # ----- post preprocessing ----- #
        telcom.fillna(telcom.mean(), inplace=True)
        return telcom
    
    
    elif dataset == "TelC":
    
        # ------ input ------ #
        # Data file opening
        sep = ","
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/TelcoCustChurn/Telco_Customer_Churn.csv'
       

        # data preprocessing
        target_col = ["Churn"]

        # data for writing file
        dataset = "TelcoCustChurn"

        telcom = pd.read_csv(dataset_name, sep)

        # --------- Unique preprocessing for this data set --------- #

        #Replacing blank spaces with null values in total charges column
        telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- Unique preprocessing for this data set --------- #

        # write report


        #Dropping null values from total charges column which contain .15% missing data 
        #telcom = telcom[telcom["TotalCharges"].notnull()]
        #telcom = telcom.reset_index()[telcom.columns]

        # replace missing values by mean there are only 11 missing values
        telcom["TotalCharges"] = pd.to_numeric(telcom["TotalCharges"])
        telcom["TotalCharges"] = telcom["TotalCharges"].fillna(telcom["TotalCharges"].mean())
        
        total_charges = telcom["TotalCharges"]

        #replace 'No internet service' to No for the following columns
        replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                        'TechSupport','StreamingTV', 'StreamingMovies']
        for i in replace_cols : 
            telcom[i]  = telcom[i].replace({'No internet service' : 'No'})

        #replace values
        telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

        #Tenure to categorical column
        def tenure_lab(telcom) :

            if telcom["tenure"] <= 12 :
                return "Tenure_0-12"
            elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
                return "Tenure_12-24"
            elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
                return "Tenure_24-48"
            elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
                return "Tenure_48-60"
            elif telcom["tenure"] > 60 :
                return "Tenure_gt_60"
        telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                              axis = 1)

        #Drop tenure column
        #telcom = telcom.drop(columns = "tenure_group",axis = 1)

        Id_col     = ['customerID']
        target_col = ["Churn"]

        cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]
        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- general preprocessing --------- #
        del telcom["customerID"]
        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)

        return telcom
    
    elif dataset == "C2C":

        # ------ input ------ #
        # Data file opening
        path = "/Churn/datasets"
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/Cell2Cell/cell2celltrain.csv'

        sep = ","

        # data preprocessing
        target_col = ["churn"]

        # data for writing file
        dataset = "Cell2Cell"

        # training dataset
        telcom = pd.read_csv(dataset_name, sep)

        # Unknown in the dataset is a NaN
        telcom.replace('Unknown', np.nan, inplace=True)
        # replace NaN by mean
        telcom.fillna(telcom.mean(), inplace=True)
        # Delete
        del telcom["CustomerID"]

        telcom.fillna(telcom.mean(), inplace=True)

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        
        return telcom
    
    elif dataset == "SATO":
        
        # ------ input ------ #
        # Data file opening
        path = "/Churn/datasets"
        #dataset_name = "south-asian/South Asian Wireless Telecom Operator (SATO 2015).csv"
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/south-asian/South Asian Wireless Telecom Operator (SATO 2015).csv'

        sep = ","

        # data preprocessing
        target_col = ["churn"]

        # data for writing file
        dataset = "south-asian"

        # ------ writing scoring report ------ #
        path_out = f"Churn/reporting/dashboard-scoring/{dataset}.csv"

        telcom = pd.read_csv(dataset_name, sep)

        # --------- Unique preprocessing for this data set --------- #

        # rename the target variable to Churn
        telcom.rename(columns={"Class":"churn"}, inplace=True)

        # change the "Churned" to 1
        telcom[target_col[0]].replace("Churned", "1", inplace=True)

        # change the active to 0
        telcom[target_col[0]].replace("Active", "0", inplace=True)

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        # --------- general preprocessing --------- #

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        
        return telcom
    

    elif dataset == "HR":
        # ------ input ------ #
        # Data file opening
        path = "/Churn/datasets"
        dataset_name = "IBM-HR/WA_Fn-UseC_-HR-Employee-Attrition.csv"
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/IBM-HR/WA_Fn-UseC_-HR-Employee-Attrition.csv'

        sep = ","

        # data preprocessing
        target_col = ["Attrition"]

        # data for writing file
        dataset = "IBM-HR"

        telcom = pd.read_csv(dataset_name, sep)

        # --------- Unique preprocessing for this data set --------- #


        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]
        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        telcom = telcom.rename(columns={'attrition': 'churn'})    
        
        return telcom
    
    elif dataset == "DSN":
        # ------ input ------ #
        # Data file opening
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/DSN-telecom-churn/TRAIN.csv'

        sep = ","

        # data preprocessing
        target_col = ["Churn Status"]

        telcom = pd.read_csv(dataset_name, sep)


        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        telcom = telcom[:-1] 

        del telcom['Customer ID']

        # works only on numerical data
        telcom.fillna(telcom.mean(), inplace=True)

        telcom['Network type subscription in Month 1'].fillna(telcom['Most Loved Competitor network in in Month 1'].mode().iloc[0], inplace=True)
        telcom['Network type subscription in Month 2'].fillna(telcom['Most Loved Competitor network in in Month 1'].mode().iloc[0], inplace=True)


        telcom['Most Loved Competitor network in in Month 1'].fillna(telcom['Most Loved Competitor network in in Month 1'].mode().iloc[0], inplace=True)
        telcom['Most Loved Competitor network in in Month 2'].fillna(telcom['Most Loved Competitor network in in Month 1'].mode().iloc[0], inplace=True)



        # remove the last obs. of the dataset, the churn value is "0.5" and it is the only one
        telcom["Churn Status"] = telcom.drop(telcom.index[-1])["Churn Status"]
        #telcom = telcom[:-1]

        multi_cols = ['Most Loved Competitor network in in Month 2',
         'Network type subscription in Month 1',
         'Network type subscription in Month 2',
         'Most Loved Competitor network in in Month 1']

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        telcom = telcom.rename(columns={'churn status': 'churn'})   
        
        return telcom
    
    elif dataset == "news":

        # ------ input ------ #
        # Data file opening
        path = "/Churn/datasets"
        dataset_name = '/home/ec2-user/SageMaker/data/churn_package/datasets/newspaper/NewspaperChurn.csv'
        sep = ","

        # data preprocessing
        target_col = ["Churn"]

        # data for writing file
        dataset = "newspaper"

        # ------ writing scoring report ------ #


        # training dataset
        telcom = pd.read_csv(dataset_name, sep)

        # set everything to lower except column
        telcom = telcom.apply(lambda x: x.astype(str).str.lower())

        # --------- Unique preprocessing for this data set --------- #

        # rename the target variable to Churn
        telcom.rename(columns={"Subscriber":"Churn"}, inplace=True)

        # change the "Churned" to 1
        telcom.Churn.replace("no", "0", inplace=True)

        # change the active to 0
        telcom.Churn.replace("yes", "1", inplace=True)

        # ----------- Categorization ----------- #

        cat_cols   = telcom.nunique()[telcom.nunique() < 10].keys().tolist()
        cat_cols   = [x for x in cat_cols if x not in target_col]

        cat_cols = list(set(cat_cols + list(telcom.select_dtypes(include=["object"]).columns)))

        num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col]

        #Binary columns with 2 values
        bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
        #Columns more than 2 values
        multi_cols = [i for i in cat_cols if i not in bin_cols]

        del telcom['Address']
        del telcom["SubscriptionID"]
        del telcom['Zip Code']

        # useless all the obs. came from CA state
        del telcom['State']

        num_cols = ['Year Of Residence', 'reward program']

        multi_cols = ['weekly fee',
         'Age range',
         'City',
         'Source Channel',
         'County',
         'Ethnicity',
         'Deliveryperiod',
         'Nielsen Prizm',
         'HH Income',
         'Language']

        telcom = preprocess(telcom.copy(), num_cols, bin_cols, multi_cols, verbose=False)
        return telcom

In [49]:
# Choose the dataset
dataset = "UCI"

# Load and preprocess the data
df = data_preprocessing(dataset)
df.head()

,international plan,voice mail plan,churn,area code_408,area code_415,area code_510,account length,number vmail messages,total day minutes,total day calls,...,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
0,0,1,0,0,1,0,0.676489,1.234883,1.566767,0.476643,...,-0.070610,-0.055940,-0.070427,0.866743,-0.465494,0.866029,-0.085008,-0.601195,-0.085690,-0.427932
1,0,1,0,0,1,0,0.149065,1.307948,-0.333738,1.124503,...,-0.108080,0.144867,-0.107549,1.058571,0.147825,1.059390,1.240482,-0.601195,1.241169,-0.427932
2,0,0,0,0,1,0,0.902529,-0.591760,1.168304,0.675985,...,-1.573383,0.496279,-1.573900,-0.756869,0.198935,-0.755571,0.703121,0.211534,0.697156,-1.188218
3,1,0,0,1,0,0,-0.428590,-0.591760,2.196596,-1.466936,...,-2.742865,-0.608159,-2.743268,-0.078551,-0.567714,-0.078806,-1.303026,1.024263,-1.306401,0.332354
4,1,0,0,0,1,0,-0.654629,-0.591760,-0.240090,0.626149,...,-1.038932,1.098699,-1.037939,-0.276311,1.067803,-0.276562,-0.049184,-0.601195,-0.045885,1.092641
